# Projection onto intersection of sets

We try to find the projection of a vector into the intersection of simple sets (whose projection can be computed easily) - See Appendix A of *A Convex Approach to Minimal Partitions Antonin Chambolle, Daniel Cremers, Thomas Pock*


$$
proj_K(x) = \bigcap_{1 \leq i_1 < i_2 \leq k} K_{i_1,i_2} \quad  K_{i_1,i_2}= \{ x: |x_{i_2} - x_{i_1}| \leq \sigma_{i1, i2} \quad \forall i1<i2 \}
$$

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

from pyproximal.projection import *
from pyproximal.proximal import *

Create vector x

In [2]:
k = 3
xtrue = np.random.normal(0, 50, k)
x = xtrue.copy()
x1 = xtrue.copy()
sigma = np.array([[3,2,1], [2,2,1], [3,4,1]])
sigma = sigma.T @ sigma
print(sigma)

for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.linalg.norm(x[i1] - x[i2]))

[[22 22  8]
 [22 24  8]
 [ 8  8  3]]
0 1 22 36.46897158479271
0 2 8 47.52425051996606
1 2 8 11.055278935173348


Projection of vector x

In [3]:
niter = 40
tol = 1e-9

x12 = np.zeros((k,k))
for iiter in range(niter):
    xold = x.copy()
    for i1 in range(k-1):
        for i2 in range(i1+1, k):
            xtilde = x[i2] - x[i1] + x12[i1, i2]
            xtildeabs = np.abs(xtilde)
            xdtilde = np.maximum(0, xtildeabs - sigma[i1, i2]) * xtilde / xtildeabs
            x[i1] = x[i1] + 0.5 * (xdtilde - x12[i1, i2])
            x[i2] = x[i2] - 0.5 * (xdtilde - x12[i1, i2])
            x12[i1, i2] = xdtilde
    if max(np.abs(x - xold)) < tol:
        break
print(iiter)

16


Check projected vector satisfy the condition

In [4]:
for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.abs(xtrue[i1] - xtrue[i2]))
        
for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.abs(x[i1] - x[i2]))

0 1 22 36.46897158479271
0 2 8 47.52425051996606
1 2 8 11.055278935173348
0 1 22 16.00000000032915
0 2 8 8.000000000329148
1 2 8 8.0


In [5]:
ic = IntersectionProj(k, 1, sigma, niter, tol)
x1 = ic(x1)
x1 - xtrue

[-6.7848559  29.68411569 40.73939462]
[False]


array([ 19.9977407 ,  -0.47123088, -19.52650982])

In [6]:
ic = Intersection(k, 1, sigma, niter, tol)
print(ic(xtrue))
x = ic.prox(xtrue, 1)
print(ic(x))

False
[-6.7848559  29.68411569 40.73939462]
[False]
True


Repeat the same, now with a matrix with n columns (algorithm works on each column indipendently)

In [7]:
k = 3
n = 5
xtrue = np.random.normal(0, 50, (k, n))
x = xtrue.copy()
x1 = xtrue.copy()
sigma = np.array([[3,2,1], [2,2,1], [3,4,1]])
sigma = sigma.T @ sigma
print(sigma)

for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.abs(x[i1] - x[i2]))

[[22 22  8]
 [22 24  8]
 [ 8  8  3]]
0 1 22 [26.26459249  7.11787694  0.2854586  15.48240221  0.68544859]
0 2 8 [53.88323769 63.17026081 13.33806176 75.2000809  81.34005829]
1 2 8 [80.14783017 56.05238386 13.05260317 59.71767869 80.65460971]


In [8]:
niter = 50
tol = 1e-5

x12 = np.zeros((k,k,n))
for iiter in range(niter):
    xold = x.copy()
    for i1 in range(k-1):
        for i2 in range(i1+1, k):
            xtilde = x[i2] - x[i1] + x12[i1, i2]
            xtildeabs = np.abs(xtilde)
            xdtilde = np.maximum(0, xtildeabs - sigma[i1, i2]) * xtilde / xtildeabs
            x[i1] = x[i1] + 0.5 * (xdtilde - x12[i1, i2])
            x[i2] = x[i2] - 0.5 * (xdtilde - x12[i1, i2])
            x12[i1, i2] = xdtilde
    if max(np.sum(np.abs(x-xold), axis=0)) < tol:
        break
print(iiter)

13


In [9]:
for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.abs(x[i1] - x[i2]))

0 1 22 [7.91384487e-07 1.52492923e-07 1.77589961e-08 1.34986927e-07
 2.68106021e-07]
0 2 8 [7.99999921 7.99999985 7.99999998 7.99999987 7.99999973]
1 2 8 [8. 8. 8. 8. 8.]


Same using the projection operator in PyProximal

In [12]:
ic = IntersectionProj(k, n, sigma, niter, tol)
x1 = ic(x1)
x1 = x1.reshape(k,n)

for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.abs(x1[i1] - x1[i2]))

0 1 22 [7.91367007e-07 1.52447738e-07 1.77413035e-08 1.34937160e-07
 2.68062820e-07]
0 2 8 [7.99999921 7.99999985 7.99999998 7.99999987 7.99999973]
1 2 8 [8. 8. 8. 8. 8.]


and the proximal operator in PyProximal

In [13]:
ic = Intersection(k, n, sigma, niter, tol)
print(ic(xtrue, 1e-3))
x = ic.prox(xtrue, 1)
print(ic(x, 1e-3))
x = x.reshape(k,n)

for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.abs(x[i1] - x[i2]))

True
True
0 1 22 [7.91367007e-07 1.52447738e-07 1.77413035e-08 1.34937160e-07
 2.68062820e-07]
0 2 8 [7.99999921 7.99999985 7.99999998 7.99999987 7.99999973]
1 2 8 [8. 8. 8. 8. 8.]
